In [2]:
from const import *

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings


from catboost import CatBoostRegressor
from catboost import Pool

warnings.simplefilter('ignore')

In [3]:
data_path = '../1.data/'
train_file_name = 'train.csv'
test_file_name  = 'test.csv'

In [4]:
def creansing_dataset(df):
    # year
    is_error_year = df['year'] > 2023
    df.loc[is_error_year, 'year'] = df[is_error_year]['year'] - 1000
    # size
    df['size'] = df['size'].replace(size_replace)
    # manufacturer
    df['manufacturer'] = df['manufacturer'].replace(manufactuer_replace)

    return df

In [5]:
train_df = pd.read_csv(f'{data_path}{train_file_name}')
test_df = pd.read_csv(f'{data_path}{test_file_name}')
master_df = pd.concat([train_df, test_df])

In [6]:
non_cols = ['id', 'price']
num_cols = ['year', 'odometer']
cat_cols = [col for col in train_df.columns if col not in non_cols + num_cols]

In [7]:
def creansing_dataset(df):
    # year
    is_error_year = df['year'] > 2023
    df.loc[is_error_year, 'year'] = df[is_error_year]['year'] - 1000
    # size
    df['size'] = df['size'].replace(size_replace)
    # manufacturer
    df['manufacturer'] = df['manufacturer'].replace(manufactuer_replace)

    return df


def create_num_features(df):
    df['elapsed_year'] = 2023 - df['year']
    del df['year']

    return df


def create_cat_features(df):
    for col in cat_cols:
        df[col] = df[col].factorize()[0]
        df[col] = df[col].astype('category')
    return df


def create_features_main(df):
    df = creansing_dataset(df)
    df = create_num_features(df)
    df = create_cat_features(df)

    return df

In [8]:
master_df = create_features_main(master_df)
train_df = master_df[~master_df['price'].isnull()].copy()
test_df  = master_df[master_df['price'].isnull()].copy()

In [9]:
def built_catboost_model(cat_features, Xt, yt, Xe, ye):
    model = \
        CatBoostRegressor(
            learning_rate = 0.1,
            iterations=300,
            loss_function = 'MAPE',
            early_stopping_rounds = 30,
            random_seed = 42
        )

    model.fit(
        X = Xt, y = yt,
        eval_set = (Xe, ye),
        cat_features = cat_features, 
        use_best_model = True, 
        verbose = 50
    )

    return model

In [10]:
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

In [11]:
train_index, valid_index = train_test_split(train_df['id'].values, test_size=0.25)

x_train = train_df.iloc[train_index].drop(columns='price')
y_train = train_df.iloc[train_index]['price']
x_valid = train_df.iloc[valid_index].drop(columns='price')
y_valid = train_df.iloc[valid_index]['price']


In [12]:
model = built_catboost_model(
    cat_cols, x_train, y_train, x_valid, y_valid
)

0:	learn: 0.5650112	test: 0.5661256	best: 0.5661256 (0)	total: 201ms	remaining: 1m
50:	learn: 0.5564281	test: 0.5591925	best: 0.5591925 (50)	total: 1.7s	remaining: 8.28s
100:	learn: 0.5536034	test: 0.5577999	best: 0.5577999 (100)	total: 3.21s	remaining: 6.33s
150:	learn: 0.5527621	test: 0.5577682	best: 0.5575974 (121)	total: 4.68s	remaining: 4.62s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.5575974251
bestIteration = 121

Shrink model to first 122 iterations.


In [13]:
mean_absolute_percentage_error(
    model.predict(x_valid), y_valid)

1.579814879301055